In [1]:
import os
import shutil
from glob import glob
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import subprocess
from copy import copy, deepcopy

import numpy as np
import nibabel as nib
from dipy.io import read_bvals_bvecs
from dipy.io.image import load_nifti, save_nifti
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel, _roll_evals
import sys

sys.path.append("../utils")
from load_neuroimaging_data import load_and_conform_image, map_wmparc2gtseg
import json

join = os.path.join

In [ ]:
with open("../splits/CNP_patients.txt", "r") as f:
    patient_ids = [line.strip() for line in f]
patient_ids = sorted(patient_ids)

# 3D Slicer Paths and other paths

In [ ]:
DWIToDTIEstimation = "/home/say26747/Downloads/Slicer-5.2.2-linux-amd64/Slicer --launch /home/say26747/Downloads/Slicer-5.2.2-linux-amd64/NA-MIC/Extensions-31382/SlicerDMRI/lib/Slicer-5.2/cli-modules/DWIToDTIEstimation"
DiffusionTensorScalarMeasurements = "/home/say26747/Downloads/Slicer-5.2.2-linux-amd64/Slicer --launch /home/say26747/Downloads/Slicer-5.2.2-linux-amd64/NA-MIC/Extensions-31382/SlicerDMRI/lib/Slicer-5.2/cli-modules/DiffusionTensorScalarMeasurements"
BRAINSFit = "/home/say26747/Downloads/Slicer-5.2.2-linux-amd64/Slicer --launch /home/say26747/Downloads/Slicer-5.2.2-linux-amd64/lib/Slicer-5.2/cli-modules/BRAINSFit"
ResampleScalarVectorDWIVolume = "/home/say26747/Downloads/Slicer-5.2.2-linux-amd64/Slicer --launch /home/say26747/Downloads/Slicer-5.2.2-linux-amd64/lib/Slicer-5.2/cli-modules/ResampleScalarVectorDWIVolume"
MNI_template_brain_only = "/hdd/HPC100_preprocess/tpl-MNI152NLin6Sym/tpl-MNI152NLin6Sym_res-01_T1w_Brain_only.nii.gz"
CNN_masking_path = "/home/say26747/Desktop/git/CNN-Diffusion-MRIBrain-Segmentation"  # look for its repository on the web

# Unring data

In [ ]:
root_data_dir = "/hdd/CNP_dataset/data/"


def worker_unring(patient_id):
    dwi_path = join(
        root_data_dir, patient_id, "dwi", f"{patient_id}_dwi.nii.gz"
    )  # change accordingly
    dwi_unring_path = join(
        root_data_dir, patient_id, "dwi", f"{patient_id}_dwi_unring"
    )  # change accordingly
    if os.path.exists(dwi_unring_path + ".nii.gz"):
        print(f"{dwi_unring_path} already exists")
        return
    # Define the Conda environment name
    conda_env_name = "fsl_base"  # Replace this with your environment name
    command = f"conda run -n {conda_env_name} unring {dwi_path} {dwi_unring_path}"
    subprocess.run(command, shell=True, executable="/bin/bash", check=True)

In [ ]:
results = Parallel(n_jobs=4)(
    delayed(worker_unring)(patient_id) for patient_id in tqdm(patient_ids)
)

# Eddy correction

In [ ]:
def worker_eddy_correction(patient_id):
    dwi_unring_path = join(
        root_data_dir, patient_id, "dwi", f"{patient_id}_dwi_unring.nii.gz"
    )
    dwi_eddy_path = join(
        root_data_dir, patient_id, "dwi", f"{patient_id}_dwi_unring_eddy"
    )
    dwi_bvals_path = join(
        root_data_dir, patient_id, "dwi", f"{patient_id}_dwi_unring.bval"
    )
    dwi_bvecs_path = join(
        root_data_dir, patient_id, "dwi", f"{patient_id}_dwi_unring.bvec"
    )
    if os.path.exists(dwi_eddy_path + ".nii.gz"):
        print(f"{dwi_eddy_path} already exists")
        return
    # Define the Conda environment name
    conda_env_name = "fsl_base"  # Replace this with your environment name
    command = f"conda run -n {conda_env_name} pnl_eddy --bvals {dwi_bvals_path} --bvecs {dwi_bvecs_path} -i {dwi_unring_path} -o {dwi_eddy_path}"
    # print(command)
    subprocess.run(command, shell=True, executable="/bin/bash")

In [ ]:
results = Parallel(n_jobs=4)(
    delayed(worker_eddy_correction)(patient_id) for patient_id in tqdm(patient_ids)
)

# Masking using masking_CNN 

In [ ]:
dwi_eddy_paths = []
for patient_id in patient_ids:
    dwi_eddy_path = dwi_eddy_path = join(
        root_data_dir, patient_id, "dwi", f"{patient_id}_dwi_unring_eddy.nii.gz"
    )
    if os.path.exists(dwi_eddy_path):
        dwi_eddy_paths.append(dwi_eddy_path)
    else:
        print(f"{dwi_eddy_path} does not exist")
print(len(dwi_eddy_paths))

# save the patient ids to a file (each path in one line)
with open("/hdd/CNP_dataset/dwi_eddy_paths.txt", "w") as f:
    for path in dwi_eddy_paths:
        f.write(path + "\n")

In [ ]:
for path in tqdm(dwi_eddy_paths):
    parent_dir = os.path.dirname(path)
    txt_file = join(parent_dir, "dwi_eddy_path.txt")
    masking_path = join(CNN_masking_path, "pipeline", "dwi_masking.py")
    model_path = join(CNN_masking_path, "model_folder")
    command = f"/home/say26747/anaconda3/bin/conda run -n dmri_seg {masking_path} -i {txt_file} -f {model_path} -filter scipy -p 97 -nproc 4"
    subprocess.run(command, shell=True, check=True, executable="/bin/bash")
    break

# diffusion-derived features

In [ ]:
def worker_dwi_calculation(patient_id):
    dest_path = f"/hdd/CNP_dataset/data/{patient_id}/DATA"
    temp_dir = "/hdd/tmp/diffusion"
    os.makedirs(dest_path, exist_ok=True)
    os.makedirs(join(temp_dir, patient_id), exist_ok=True)
    dwi_root_path = f"/hdd/CNP_dataset/data/{patient_id}/dwi"
    if os.path.exists(join(dest_path, "trace_from_eigs.nii.gz")):
        print(f"Already processed {patient_id}")
        return
    # load the bvals and bvecs
    bvals, bvecs = read_bvals_bvecs(
        join(dwi_root_path, f"{patient_id}_dwi_unring_eddy.bval"),
        join(dwi_root_path, f"{patient_id}_dwi_unring_eddy.bvec"),
    )
    # load the DWI data
    dwi_data, dwi_affine = load_nifti(
        join(dwi_root_path, f"{patient_id}_dwi_unring_eddy.nii.gz")
    )

    b_value_threshold = 50
    b0_indecies = np.where(bvals < b_value_threshold)[0]
    b1000_indecies = np.where(np.abs(bvals - 1000) < b_value_threshold)[0]
    combined_indecies = np.concatenate([b0_indecies, b1000_indecies])
    dwi_filtered_data = dwi_data[..., combined_indecies]
    bvals_filtered = bvals[combined_indecies]
    bvecs_filtered = bvecs[combined_indecies]
    # save the filtered data
    save_nifti(join(temp_dir, patient_id, "DWI.nii.gz"), dwi_filtered_data, dwi_affine)
    # save the filtered bvals and bvecs
    np.savetxt(join(temp_dir, patient_id, "bvals"), bvals_filtered)
    np.savetxt(join(temp_dir, patient_id, "bvecs"), bvecs_filtered)
    # now write tell where to save the nhdr file
    dwi_nhdr = join(temp_dir, patient_id, "DWI_nhdr.nhdr")
    mask_nhdr = join(temp_dir, patient_id, "mask_nhdr.nhdr")
    subprocess.run(
        [
            "nhdr_write.py",
            "--nifti",
            join(temp_dir, patient_id, "DWI.nii.gz"),
            "--bval",
            join(temp_dir, patient_id, "bvals"),
            "--bvec",
            join(temp_dir, patient_id, "bvecs"),
            "--nhdr",
            dwi_nhdr,
        ]
    )
    shutil.copy(
        join(dwi_root_path, f"{patient_id}_dwi_unring_eddy_bse-multi_BrainMask.nii.gz"),
        join(temp_dir, patient_id, "mask.nii.gz"),
    )
    subprocess.run(
        [
            "nhdr_write.py",
            "--nifti",
            join(temp_dir, patient_id, "mask.nii.gz"),
            "--nhdr",
            mask_nhdr,
        ]
    )
    dti_nhdr = join(temp_dir, patient_id, "DTI.nhdr")
    b0_nhdr = join(temp_dir, patient_id, "b0.nhdr")
    subprocess.run(
        DWIToDTIEstimation.split()
        + [
            "--enumeration",
            "LS",
            dwi_nhdr,
            dti_nhdr,
            b0_nhdr,
            "-m",
            mask_nhdr,
        ]
    )
    fa_nhdr = join(temp_dir, patient_id, "FA.nhdr")
    trace_nhdr = join(temp_dir, patient_id, "trace.nhdr")
    minEig_nhdr = join(temp_dir, patient_id, "minEig.nhdr")
    midEig_nhdr = join(temp_dir, patient_id, "midEig.nhdr")
    maxEig_nhdr = join(temp_dir, patient_id, "maxEig.nhdr")
    planarity_nhdr = join(temp_dir, patient_id, "planarity.nhdr")
    linearity_nhdr = join(temp_dir, patient_id, "linearity.nhdr")
    sphericity_nhdr = join(temp_dir, patient_id, "sphericity.nhdr")

    fa_nifti = join(dest_path, "FA.nii.gz")
    trace_nifti = join(dest_path, "trace.nii.gz")
    minEig_nifti = join(dest_path, "minEig.nii.gz")
    midEig_nifti = join(dest_path, "midEig.nii.gz")
    maxEig_nifti = join(dest_path, "maxEig.nii.gz")
    planarity_nifti = join(dest_path, "planarity.nii.gz")
    linearity_nifti = join(dest_path, "linearity.nii.gz")
    sphericity_nifti = join(dest_path, "sphericity.nii.gz")

    subprocess.run(
        DiffusionTensorScalarMeasurements.split()
        + [
            "--enumeration",
            "FractionalAnisotropy",
            dti_nhdr,
            fa_nhdr,
        ]
    )
    subprocess.run(
        DiffusionTensorScalarMeasurements.split()
        + [
            "--enumeration",
            "Trace",
            dti_nhdr,
            trace_nhdr,
        ]
    )
    subprocess.run(
        DiffusionTensorScalarMeasurements.split()
        + [
            "--enumeration",
            "MinEigenvalue",
            dti_nhdr,
            minEig_nhdr,
        ]
    )
    subprocess.run(
        DiffusionTensorScalarMeasurements.split()
        + [
            "--enumeration",
            "MidEigenvalue",
            dti_nhdr,
            midEig_nhdr,
        ]
    )
    subprocess.run(
        DiffusionTensorScalarMeasurements.split()
        + [
            "--enumeration",
            "MaxEigenvalue",
            dti_nhdr,
            maxEig_nhdr,
        ]
    )
    subprocess.run(
        DiffusionTensorScalarMeasurements.split()
        + [
            "--enumeration",
            "PlanarMeasure",
            dti_nhdr,
            planarity_nhdr,
        ]
    )
    subprocess.run(
        DiffusionTensorScalarMeasurements.split()
        + [
            "--enumeration",
            "LinearMeasure",
            dti_nhdr,
            linearity_nhdr,
        ]
    )
    subprocess.run(
        DiffusionTensorScalarMeasurements.split()
        + [
            "--enumeration",
            "SphericalMeasure",
            dti_nhdr,
            sphericity_nhdr,
        ]
    )

    subprocess.run(
        [
            "nifti_write.py",
            "-i",
            fa_nhdr,
        ]
    )
    subprocess.run(
        [
            "nifti_write.py",
            "-i",
            trace_nhdr,
        ]
    )
    subprocess.run(
        [
            "nifti_write.py",
            "-i",
            minEig_nhdr,
        ]
    )
    subprocess.run(
        [
            "nifti_write.py",
            "-i",
            midEig_nhdr,
        ]
    )
    subprocess.run(
        [
            "nifti_write.py",
            "-i",
            maxEig_nhdr,
        ]
    )
    subprocess.run(
        [
            "nifti_write.py",
            "-i",
            planarity_nhdr,
        ]
    )
    subprocess.run(
        [
            "nifti_write.py",
            "-i",
            linearity_nhdr,
        ]
    )
    subprocess.run(
        [
            "nifti_write.py",
            "-i",
            sphericity_nhdr,
        ]
    )
    subprocess.run(
        [
            "nifti_write.py",
            "-i",
            b0_nhdr,
        ]
    )
    # now move the needed files
    shutil.move(fa_nhdr.replace(".nhdr", ".nii.gz"), fa_nifti)
    shutil.move(trace_nhdr.replace(".nhdr", ".nii.gz"), trace_nifti)
    shutil.move(minEig_nhdr.replace(".nhdr", ".nii.gz"), minEig_nifti)
    shutil.move(midEig_nhdr.replace(".nhdr", ".nii.gz"), midEig_nifti)
    shutil.move(maxEig_nhdr.replace(".nhdr", ".nii.gz"), maxEig_nifti)
    shutil.move(planarity_nhdr.replace(".nhdr", ".nii.gz"), planarity_nifti)
    shutil.move(linearity_nhdr.replace(".nhdr", ".nii.gz"), linearity_nifti)
    shutil.move(sphericity_nhdr.replace(".nhdr", ".nii.gz"), sphericity_nifti)

    shutil.move(b0_nhdr.replace(".nhdr", ".nii.gz"), join(dest_path, "b0.nii.gz"))
    shutil.copy(
        join(dwi_root_path, f"{patient_id}_dwi_unring_eddy_bse-multi_BrainMask.nii.gz"),
        join(dest_path, "brain_mask.nii.gz"),
    )

    shutil.rmtree(join(temp_dir, patient_id))

    img_data, img_affine = load_nifti(
        join(os.path.dirname(dwi_root_path), "freesurfer", "mri", "wmparc.mgz")
    )
    save_nifti(join(dest_path, "wmparc.nii.gz"), img_data, img_affine)
    img_data, img_affine = load_nifti(
        join(os.path.dirname(dwi_root_path), "freesurfer", "mri", "T1.mgz")
    )
    save_nifti(join(dest_path, "T1.nii.gz"), img_data, img_affine)

    eigs_data = []
    eigs_affine = []
    for eig in ["minEig", "midEig", "maxEig"]:
        eig_path = join(dest_path, f"{eig}.nii.gz")
        eig_data, eig_affine = load_nifti(eig_path)
        eigs_data.append(eig_data)
        eigs_affine.append(eig_affine)
    save_nifti(
        join(dest_path, "trace_from_eigs.nii.gz"),
        np.sum(eigs_data, axis=0),
        eigs_affine[0],
    )
    save_nifti(
        join(dest_path, "MD_from_eigs.nii.gz"),
        np.mean(eigs_data, axis=0),
        eigs_affine[0],
    )

In [ ]:
results = Parallel(n_jobs=5)(
    delayed(worker_dwi_calculation)(patient_id) for patient_id in tqdm(patient_ids)
)

# wmparc to dwi

In [ ]:
def worker_fs2dwi(patient_id):
    dwi_path = (
        f"/hdd/CNP_dataset/data/{patient_id}/dwi/{patient_id}_dwi_unring_eddy.nii.gz"
    )
    dwi_mask_path = f"/hdd/CNP_dataset/data/{patient_id}/dwi/{patient_id}_dwi_unring_eddy_bse-multi_BrainMask.nii.gz"
    bse_path = f"/hdd/CNP_dataset/data/{patient_id}/dwi/{patient_id}_dwi_unring_eddy_bse.nii.gz"
    bval_path = (
        f"/hdd/CNP_dataset/data/{patient_id}/dwi/{patient_id}_dwi_unring_eddy.bval"
    )
    freesurfer_path = f"/hdd/CNP_dataset/data/{patient_id}/freesurfer"
    dwi_root_path = f"/hdd/CNP_dataset/data/{patient_id}/dwi"
    fs2dwi_exc = "nifti_fs2dwi"

    command = f"{fs2dwi_exc} --dwi {dwi_path} --dwimask {dwi_mask_path} --bse {bse_path} --bvals {bval_path} -f {freesurfer_path} -o {dwi_root_path} direct"
    subprocess.run(command, shell=True, check=True)

In [ ]:
results = Parallel(n_jobs=5)(
    delayed(worker_fs2dwi)(patient_id) for patient_id in tqdm(patient_ids)
)

# Making the data registered to MNI and conformed

In [ ]:
with open("../utils/wmparc_values_to_sequence.json", "r") as file:
    lookup_table = json.load(file)
# swap the keys and values
lookup_table = {v: k for k, v in lookup_table.items()}


def worker_ras_MNI_conform(patient_id):
    root_path = f"/hdd/CNP_dataset/data/{patient_id}"
    wmparc_dwi_path = join(root_path, "dwi", "wmparcInDwi.nii.gz")
    dwi_mask_path = join(root_path, "DATA", "brain_mask.nii.gz")
    b0 = join(root_path, "DATA", "b0.nii.gz")
    # modalities=["FA", "trace_from_eigs", "maxEig", "minEig", "midEig"]
    modalities = ["sphericity"]
    img_path = join(root_path, "DATA", "images")
    os.makedirs(img_path, exist_ok=True)
    labels_path = join(root_path, "DATA", "labels")
    os.makedirs(labels_path, exist_ok=True)

    for modality in modalities:
        modality_path = join(root_path, "DATA", f"{modality}.nii.gz")
        img = nib.load(modality_path)
        img = img.as_reoriented(nib.orientations.io_orientation(img.affine))
        nib.save(img, join(img_path, f"{modality}_ras.nii.gz"))

        ResampleScalarVectorDWIVolume_command = (
            ResampleScalarVectorDWIVolume.split()
            + [
                "--interpolation",
                "linear",
                "--Reference",
                MNI_template_brain_only,
                "--transformationFile",
                join(root_path, "DATA", "transformations", "b0_to_MNI_Brain.tfm"),
                join(img_path, f"{modality}_ras.nii.gz"),
                join(img_path, f"{modality}_ras_MNI.nii.gz"),
            ]
        )
        subprocess.run(ResampleScalarVectorDWIVolume_command)

        modality_path = join(img_path, f"{modality}_ras_MNI.nii.gz")
        header_info, affine_info, data = load_and_conform_image(
            modality_path, interpol=3, imagetype="image"
        )
        nib.save(
            nib.Nifti1Image(data, affine=affine_info, header=header_info),
            modality_path.replace(".nii.gz", "_conform.nii.gz"),
        )

In [ ]:
results = Parallel(n_jobs=5)(
    delayed(worker_ras_MNI_conform)(patient_id) for patient_id in tqdm(patient_ids)
)